In [1]:
import sys
sys.path.append('/afs/ee.cooper.edu/user/t/a/tam8/documents/ml_misc/semi_supervised/')
import coreg
reload(coreg)
import trireg
reload(trireg)
sys.path.append('/afs/ee.cooper.edu/user/t/a/tam8/documents/ml_misc/ordinal/')
import simple
reload(simple)

import pandas as pd
import numpy as np
from sklearn import preprocessing
import xgboost as xgb

import transformers as tforms
reload(tforms)
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion, make_union

import metrics
reload(metrics)
from sklearn.cross_validation import StratifiedKFold

from sklearn.base import clone

from sklearn.svm import SVC, LinearSVC, SVR
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, ElasticNet, Ridge, Lasso, SGDRegressor
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.isotonic import IsotonicRegression

from sklearn.preprocessing import StandardScaler

from collections import Counter
import minirank as mr

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from time import time

def wgmean(x, w):
    return np.exp(np.sum(w*np.log(x), axis=1) / np.sum(w, axis=1))

In [2]:
# Load Data
import cPickle as pickle
train_pd  = pd.read_pickle('saved/train_pd_enc.p')
test_pd  = pd.read_pickle('saved/test_pd_enc.p')
labels = pd.read_pickle('saved/labels.p')
test_ind = pickle.load(open('saved/test_ind.p'))
y_binned = np.load('saved/y_binned.npy')


In [3]:
train = np.array(train_pd)
test = np.array(test_pd)

X_train = train.astype(float)
X_test = test.astype(float)
y_train = np.array(labels)

pipe_x = make_pipeline(
    make_union(
        tforms.IdentityTformer(),
#         make_pipeline(AddTformer(1), BoxCoxTformer()),
#         AnscombeTformer(),
    ),
    StandardScaler(),
)
pipe_y = make_pipeline(
#     tforms.IdentityTformer(),
#     tforms.BoxCoxTformer(),
#     tforms.LogTfortforms.mer(),
#     tforms.AnscombeTformer(),
#     tforms.FreemanTukeyTformer(),
#     tforms.ArcsinhTformer(),
    StandardScaler(),
    
)
pipe_x.fit(np.r_[X_train, X_test])


X_train = pipe_x.transform(X_train)
X_test = pipe_x.transform(X_test)
# y_train = pipe_y.fit_transform(y_train)

# small_n = 5000
# X_train = X_train[:small_n,:]
# y_train = y_train[:small_n]
# y_binned = y_binned[:small_n]

print y_train.shape
print X_train.shape
print X_test.shape
print len(np.unique(y_train))
print len(np.unique(y_binned))

(50999,)
(50999, 111)
(51000, 111)
50
6


In [4]:
print y_train[:50]
print y_binned[:50]

[ 1  4  1  1  1  1  5  1  1  1  7 15  1  4 14  1  1  1  4  1  4  4  8 16  5
  1  6  7  1  1  1  7  5  1  6  4  1  1  8  1  1  4  7  5  1  4  4  4  1  1]
[1 3 1 1 1 1 3 1 1 1 4 5 1 3 5 1 1 1 3 1 3 3 4 5 3 1 3 4 1 1 1 4 3 1 3 3 1
 1 4 1 1 3 4 3 1 3 3 3 1 1]


# Keras setup

In [4]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, MaxoutDense, Reshape
from keras.layers.advanced_activations import PReLU
from keras.layers.normalization import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adadelta, Adagrad, Adam, RMSprop
from keras.utils import np_utils

losses = []
from keras.callbacks import Callback
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        pass
        
    def on_epoch_end(self, epoch, logs={}):
        self.losses.append(logs.get('val_loss'))
    

Using gpu device 0: GeForce GTX 580


In [6]:
# binary classification model
model = Sequential()

hidden_size = 256

model.add(Dense(111, hidden_size))
model.add(Dropout(0.2))
# model.add(Activation('relu'))
# model.add(PReLU(hidden_size))
model.add(MaxoutDense(hidden_size, hidden_size, 2))
model.add(BatchNormalization(hidden_size))
model.add(Dropout(0.5))
model.add(Dense(hidden_size, hidden_size))
# model.add(Activation('relu'))
# model.add(PReLU(hidden_size))
model.add(MaxoutDense(hidden_size, hidden_size, 2))
model.add(BatchNormalization(hidden_size))
model.add(Dropout(0.5))
model.add(Dense(hidden_size, hidden_size))
# model.add(Activation('relu'))
# model.add(PReLU(hidden_size))
model.add(MaxoutDense(hidden_size, hidden_size, 2))
model.add(BatchNormalization(hidden_size))
model.add(Dropout(0.5))
model.add(Dense(hidden_size, 2))

# model.compile(loss='binary_crossentropy', 
#               optimizer='rmsprop', 
#               class_mode='binary')
# model.save_weights('saved/nn_weights', overwrite=True)

nn_params = {'model': model,
             'optimizer': 'rmsprop',
             'loss': 'binary_crossentropy'}
clf_nn = KerasClassifier(**nn_params)

label_bin_tform = lambda y: np_utils.to_categorical(y, 2)

In [10]:
gbm = xgb.XGBRegressor(
    objective="reg:linear",
    n_estimators=600,
    learning_rate=0.01,
#     gamma=0.0,
    max_depth=8,
    min_child_weight=5,
#     max_delta_step=10,
    subsample=0.8,
    colsample_bytree=0.8,
#     scale_pos_weight=1.0,
    base_score=0.5,
    nthread=7,
    seed=np.random.randint(0,100),
    silent=True,
)

gbm_bin = xgb.XGBClassifier(
    objective="binary:logistic",
    n_estimators=500,
    learning_rate=0.01,
#     gamma=0.0,
    max_depth=8,
    min_child_weight=5,
#     max_delta_step=10,
    subsample=0.8,
    colsample_bytree=0.8,
#     scale_pos_weight=1.0,
    base_score=0.5,
    nthread=8,
    seed=np.random.randint(0,100),
    silent=True,
)



# clf0 = simple.SimpleOrdinalClassifier(gbm_bin, n_jobs=1)
# clf0 = simple.SimpleOrdinalClassifier(GaussianNB(), n_jobs=1)
# clf0 = simple.SimpleOrdinalClassifier(base_estimator_type=GaussianNB, base_estimator_params={}, n_jobs=1)
# clf0 = simple.SimpleOrdinalClassifier(SVC(probability=True), n_jobs=8)
# clf0 = simple.SimpleOrdinalClassifier(clf_nn, n_jobs=1)
clf0 = simple.SimpleOrdinalClassifier(base_estimator_type=KerasClassifier,
                                      base_estimator_params=nn_params,
                                      label_transformer=label_bin_tform,
                                      n_jobs=1)


# clfs = [clf0, gbm]
clfs = [LinearRegression()]
# clfs = [clf1, clf2, ]


# h = [clone(clf1), clone(clf2)]
# clf = coreg.CoReg(h=h, T=5, verbose=True, n_jobs=1)

scores = []
scores_base = []
lols = []
n_reps = 1
k = 3
for reps in range(n_reps):
    skf = StratifiedKFold(y_train, n_folds=k,
                          shuffle=True,
                          random_state=np.random.randint(0,100))
    for ii, (train, valid) in enumerate(skf):
        
        
#         history = LossHistory()   # for keras
        print 'Fold %d' % ii,
        X_train_k = X_train[train]
        X_valid_k = X_train[valid]
        y_train_k = y_train[train]
        y_valid_k = y_train[valid]
        y_train_binned_k = y_binned[train]
        y_valid_binned_k = y_binned[valid]     
        
        
        X_all = np.r_[X_train_k, X_valid_k]
        y_all = np.r_[y_train_k, np.nan*np.ones(len(y_valid_k))]
        
        tic = time()
#         clf.fit(X_train_k, y_train_k)
#         clf.fit(X_all, y_all)
        
        for clf in clfs:
#             clf.fit(X_train_k, y_train_k)
#             clf.fit(X_train_k, y_train_binned_k)
            clf.fit(X_train_k, y_train_binned_k,
                   nb_epoch=60, batch_size=1024*16, )
    
    
        
        # Minirank
#         w, theta = mr.ordinal_logistic_fit(X_train_k, y_train_k, verbose=False,
#                                 solver='TNC')

        
        toc = time() - tic
        print 'Train time: %2.3f s\t' % toc,
        tic = time()

        
        
#         valid_preds = [clf.predict_weighted(X_valid_k,) 
#                        if hasattr(clf, 'predict_weighted') 
#                        else clf.predict(X_valid_k)
#                        for clf in clfs]
        
        valid_preds = [clf.predict_weighted(X_valid_k, batch_size=1024*16) 
               if hasattr(clf, 'predict_weighted') 
               else clf.predict(X_valid_k)
               for clf in clfs]
        
        try:
            lol = metrics.normalized_gini(y_valid_k, 
                                          clfs[0].predict_weighted(X_valid_k, geometric=True))
            lols.append(lol)
        except:
            pass

        # Minirank
#         valid_preds = mr.ordinal_logistic_predict(w, theta, X_valid_k)
        
        valid_base_preds = np.mean([clf.predict(X_valid_k) for clf in clfs
                                   ], 
                                   axis=0)

        
        score = [metrics.normalized_gini(y_valid_k, v) for v in valid_preds]
        score_base = metrics.normalized_gini(y_valid_k, valid_base_preds)
        
        toc_pred = time() - tic
        print 'Pred time: %2.3f s\t' % toc_pred, 
        
        print 'Score', score
        print 'Score avg ensemble', score_base
        scores.append(score)
        scores_base.append(score_base)
        
#         break
        
        
print "done"
print 'Score:', np.array(scores).mean(axis=0)
print 'Base Score:', np.array(scores_base).mean()
print 'lol', lols, np.mean(lols)

 Fold 0 Train time: 0.250 s	Pred time: 0.269 s	Score [0.3143241579939952]
Score avg ensemble 0.314324157994
Fold 1 Train time: 0.211 s	Pred time: 0.266 s	Score [0.3332024658469817]
Score avg ensemble 0.333202465847
Fold 2 Train time: 0.220 s	Pred time: 0.265 s	Score [0.33503433745908395]
Score avg ensemble 0.335034337459
done
Score: [ 0.32752032]
Base Score: 0.327520320433
lol [] nan


/usr/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


In [16]:
q = [500, 1000, 5000, 10000]
t_train = [0.68, 1.9, 27, ]
t_pred = [0.32, 1.25, 27, ]

# gbm is ~ 0.381  @ 30 sec per fold?
# gbm on binned labels ~ 0.378 @ 30 sec per fold... ~_~
# ordinal nn binned is like 0.34 with stdscaling X 60 epochs


0.267811596932
0.293322383929


In [ ]:
print scores
print scores_base
print lol
# print scores2

In [25]:
metrics.normalized_gini(y_valid_k, clfs[0].predict_weighted(X_valid_k, geometric=True))

0.37712908899661984

In [32]:
params = {}
params["objective"] = "reg:linear"
params["eta"] = 0.01
params["min_child_weight"] = 5
params["subsample"] = 0.8
params["colsample_bytree"] = 0.8
# params["max_delta_step"] = 10
params["silent"] = 1
params["max_depth"] = 8 #7
params["scale_pos_weight"] = 1.0
# params["gamma"] = 0


plst = list(params.items())

max_rounds = 1000
xgtest = xgb.DMatrix(X_test)

#create a train and validation dmatrices 
xgtrain = xgb.DMatrix(X_train_k, label=y_train_k)
xgval = xgb.DMatrix(X_valid_k, label=y_valid_k)

#train using early stopping and predict
watchlist = [(xgtrain, 'train'),(xgval, 'val')]
model = xgb.train(plst, xgtrain, max_rounds, watchlist, early_stopping_rounds=20)
# preds1 = model.predict(xgtest)


Will train until val error hasn't decreased in 20 rounds.
[0]	train-rmse:5.307869	val-rmse:5.343077
[1]	train-rmse:5.281321	val-rmse:5.318596
[2]	train-rmse:5.255945	val-rmse:5.294526
[3]	train-rmse:5.230171	val-rmse:5.270779
[4]	train-rmse:5.204879	val-rmse:5.247344
[5]	train-rmse:5.179912	val-rmse:5.223663
[6]	train-rmse:5.155277	val-rmse:5.201127
[7]	train-rmse:5.131289	val-rmse:5.178530
[8]	train-rmse:5.106847	val-rmse:5.155623
[9]	train-rmse:5.083262	val-rmse:5.133870
[10]	train-rmse:5.060019	val-rmse:5.112049
[11]	train-rmse:5.036912	val-rmse:5.090539
[12]	train-rmse:5.014286	val-rmse:5.069273
[13]	train-rmse:4.992133	val-rmse:5.048791
[14]	train-rmse:4.970232	val-rmse:5.028564
[15]	train-rmse:4.948495	val-rmse:5.008514
[16]	train-rmse:4.926886	val-rmse:4.988660
[17]	train-rmse:4.905847	val-rmse:4.968964
[18]	train-rmse:4.885324	val-rmse:4.950142
[19]	train-rmse:4.864790	val-rmse:4.931548
[20]	train-rmse:4.844736	val-rmse:4.913131
[21]	train-rmse:4.824436	val-rmse:4.894755
[22]	t

# Generate submission

In [9]:
preds = pd.DataFrame({"Id": test_ind, "Hazard": preds})
preds = preds.set_index('Id')
preds.to_csv('xgboost_benchmark.csv')

array([[ 15.,   3.,   2., ...,   4.,   2.,   2.],
       [ 16.,  14.,   5., ...,   4.,   2.,   1.],
       [ 10.,  10.,   5., ...,   4.,   6.,   1.],
       ..., 
       [ 18.,   7.,   5., ...,   4.,   1.,   1.],
       [ 18.,  17.,   5., ...,   2.,   2.,   6.],
       [  5.,  15.,   3., ...,   4.,   5.,   4.]])